In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import pymongo
from webdriver_manager.chrome import ChromeDriverManager
from pprint import pprint

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [/Users/darren/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache


# NASA Mars News

In [3]:
url_Mars = 'https://mars.nasa.gov/news/'
browser.visit(url_Mars)

In [4]:
html = browser.html
soup_Mars = BeautifulSoup(html, 'html.parser') # Parse HTML results with Soup
#print(soup_Mars)

In [5]:
results_Mars = soup_Mars.select_one("ul.item_list li.slide")
results_Mars.find("div", class_="content_title")

<div class="content_title"><a href="/news/8936/nasas-ingenuity-helicopter-to-begin-new-demonstration-phase/" target="_self">NASA's Ingenuity Helicopter to Begin New Demonstration Phase</a></div>

In [6]:
# Scrape the latest news title
title_Mars = results_Mars.find("div", class_="content_title").get_text()
print(f'The latest news title is: {title_Mars}' )

The latest news title is: NASA's Ingenuity Helicopter to Begin New Demonstration Phase


In [7]:
# Scrape the latest paragraph text
para_Mars = results_Mars.find("div", class_="article_teaser_body").get_text()
print(para_Mars)

The Red Planet rotorcraft will shift focus from proving flight is possible on Mars to demonstrating flight operations that future aerial craft could utilize.


# JPL Mars Space Images - Featured Image

In [8]:
url_JPL = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url_JPL)

In [9]:
html_JPL = browser.html
soup_JPL = BeautifulSoup(html_JPL, 'html.parser') # Parse HTML results with Soup
#print(soup_Mars)

In [10]:
full_img = soup_JPL.find("div", class_="header")

In [11]:
image_url = full_img.find("img", class_="headerimage fade-in").get("src")
# Use base URL to obtain absolute URL
print(f'the featured imgae url:')
print(f"https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{image_url}")

the featured imgae url:
https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg


# Mars Facts

In [12]:
fact_url='https://space-facts.com/mars/'
Mars_table = pd.read_html(fact_url)
pprint(Mars_table)

[                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
0               Diameter:         6,779 km        12,742 km
1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
2                  Moons:                2                1
3      Distance from Sun:   227,943,824 km   149,598,262 km
4         Length of Year:   687 Earth days      365.24 days
5            Temperature:     -87 to -5 °C      -88 to 58°C,
                       0  

In [13]:
type(Mars_table)

list

In [14]:
table_df = Mars_table[1]
table_df.columns = ["Mars - Earth Comparison", "Mars", "Earth"]

In [15]:
table_df.head()

,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days


### Cleanup of extra rows

In [16]:
table_df = table_df.iloc[0:]
table_df.head()

,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days


Set the index to the Mars - Earth Comparison

In [17]:
table_df.set_index('Mars - Earth Comparison', inplace=True)
table_df.head()

,Mars,Earth
Mars - Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days


### DataFrames as HTML

In [18]:
html_table = table_df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</table>'

### Strip unwanted newlines to clean up the table

In [19]:
html_table.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars</th>      <th>Earth</th>    </tr>    <tr>      <th>Mars - Earth Comparison</th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Diameter:</th>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>Moons:</th>      <td>2</td>      <td>1</td>    </tr>    <tr>      <th>Distance from Sun:</th>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <th>Length of Year:</th>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <th>Temperature:</th>      <td>-87 to -5 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>'

In [20]:
table_df.to_html('Mars_Facts_table.html')

In [21]:
# To open the html table in a browser
!open Mars_Facts_table.html

# Hemisphere of Mars

In [22]:
hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphere_url)

In [23]:
hemi_html = browser.html
hemi_soup = BeautifulSoup(hemi_html, 'html.parser')

In [24]:
results = hemi_soup.find_all("div", class_='item')
# Loop over result to get title and image url of products
for result in results:
    # Error handling
    try:
        img_hemi = result.find('a', class_='itemLink product-item')
        img_url_hemi = img_hemi.find('img', class_='thumb').get("src")
    
        title_hemi = result.find('div', class_='description')
        title_text = title_hemi.a.h3.text
        if (img_hemi and title_text):
            print('-----------------------')
            print(f'Image URL:')
            print(f'https://astrogeology.usgs.gov{img_url_hemi}')
            print(f'Title:')
            print(title_text)
    except AttributeError as Error:
        print(Error)

-----------------------
Image URL:
https://astrogeology.usgs.gov/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png
Title:
Cerberus Hemisphere Enhanced
-----------------------
Image URL:
https://astrogeology.usgs.gov/cache/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png
Title:
Schiaparelli Hemisphere Enhanced
-----------------------
Image URL:
https://astrogeology.usgs.gov/cache/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png
Title:
Syrtis Major Hemisphere Enhanced
-----------------------
Image URL:
https://astrogeology.usgs.gov/cache/images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png
Title:
Valles Marineris Hemisphere Enhanced
